In [69]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from math import *
from astroML.time_series import generate_damped_RW
import argparse
import random
#%matplotlib qt
%matplotlib inline
import sys
np.set_printoptions(threshold=sys.maxsize)
import pickle

In [70]:
data = np.loadtxt('RXJ1131_ALL.rdb',skiprows=2)
data1=np.genfromtxt('HE0435_ECAM.rdb',skip_header=2)
time = np.sort(np.concatenate((data[:,0],data1[:,0])))
t_drive_new = np.loadtxt('t_drive_new.txt')

In [71]:
'''simulating data using the sampling about as the time axis'''

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--time_initial'  , '-ti  ', type=float, default=-365.        ,help='Set the initial time (default: t=-365 [day])')
parser.add_argument('--time_final'    , '-tf  ', type=float, default=365.         ,help='Set the final time (default: t=365 [day])')
parser.add_argument('--delta_time'    , '-dt  ', type=float, default=0.1          ,help='Set the time interval (default: dt=0.1 [day])')
parser.add_argument('--fname'         , '-fn  ', type=str  , default='LC_DRW.txt' ,help='Set the output filename (default: LC_DRW.txt)')

parser.add_argument('--tau'           , '-tau ', type=float, default=300          ,help='Set the relaxation time (default: tau=300)')
parser.add_argument('--strc_func_inf' , '-sf  ', type=float, default=0.1          ,help='Set the structure function at infinity (default: SFinf=0.1)')
parser.add_argument('--xmean'         , '-xmu ', type=float, default=1.           ,help='Set the mean value of random walk (default: Xmean=1.)')
parser.add_argument('--ran_seed'      , '-sn  ', type=str  , default='123'        ,help='Set the random seed (r: random, snnn: random seed)')
parser.add_argument('--redshift_src'  , '-zs'  , type=float, default=0.5          ,help='Set the redshift of source (default: zs=0.5)')

parser.add_argument('--target_dir'    , '-td'  ,             default='.',          help='Set the output directory')


args = parser.parse_args()

ti = args.time_initial
tf = args.time_final
dt = args.delta_time
fn = args.fname

xmean = args.xmean
tau = args.tau
SFinf = args.strc_func_inf
zs = args.redshift_src
sn = args.ran_seed

stem_out = args.target_dir


if (sn == 'r'):
    np.random.seed()
else:
    sn = int(sn)
    np.random.seed(sn)


In [72]:
sn = np.random.randn(1)
time_delay = np.random.rand(4)*15 # time delays: these are large, just to see effects throughout process

t_drive = t_drive_new
f_drive = generate_damped_RW(t_drive, tau=tau, z=zs, SFinf=SFinf, xmean=xmean,random_state=5)

### Part 3

In [73]:
# 0th image starts from 0 to 1368 in indices
# 1st image starts from 1369 to 2737 in indices
# 2nd image starts from 2738 to 4106 in indices
# 3rd image starts from 4106 to 5475 in indices
# true image starts from 5475 to 6844 in indices

In [74]:
# concatenating all the shifted time samples
sample = np.concatenate((t_drive+time_delay[0],t_drive+time_delay[1],t_drive+time_delay[2],t_drive+time_delay[3]))#t_drive))

In [75]:
# gives the indices of time shifted samples within the sorted, concatenated array
'''all that needs to be understood from this is that the sanity check will work, uncomment 2nd line of cell below to try'''
index0 = []
for i in range(1369):
    index0.append(np.where(np.argsort(sample) == i))
index0 = np.array(index0).flatten()

index1 = []
for i in range(1369,2738):
    index1.append(np.where(np.argsort(sample) == i))
index1 = np.array(index1).flatten()

index2 = []
for i in range(2738,4107):
    index2.append(np.where(np.argsort(sample) == i))
index2 = np.array(index2).flatten()

index3 = []
for i in range(4107,5476):
    index3.append(np.where(np.argsort(sample) == i))
index3 = np.array(index3).flatten()

index_truth = []
for i in range(5476,6845):
    index_truth.append(np.where(np.argsort(sample) == i))
index_truth = np.array(index_truth).flatten()

In [76]:
'''sanity check, replace 3 with 1,0,2, finding the time shifted samples that are distributed 
in the sorted concatenated array'''
#np.sort(sample)[index3] == (t_drive+time_delay[3])

'sanity check, replace 3 with 1,0,2, finding the time shifted samples that are distributed \nin the sorted concatenated array'

In [77]:
# generating magnitudes
f = generate_damped_RW(np.sort(sample), tau=tau, z=zs, SFinf=SFinf, xmean=xmean,random_state=5)
# finding the magnitudes corresponding to each time shifted sample
f_t0 = f[index0]
f_t1 = f[index1]
f_t2 = f[index2]
f_t3 = f[index3]
f_truth = f[index_truth]

In [78]:
%matplotlib qt

#plt.plot(t_drive, f_truth, 'o', color='black', label='DRW- True')
plt.plot(t_drive, f_t0+0.3, 'o', label='DRW1')  
plt.plot(t_drive, f_t1+0.6, 'o',  label='DRW2')  
plt.plot(t_drive ,f_t2+0.9, 'o', label='DRW3')  
plt.plot(t_drive, f_t3+0.12, 'o', label='DRW4')  

plt.xlabel('t (days)')
plt.ylabel('Fraction Variation')
plt.legend()
plt.show()

In [57]:
%matplotlib qt

#plt.plot(t_drive, f_truth, color='black', label='DRW- True')
plt.plot(t_drive+time_delay[0], f_t0, 'o', label='DRW1')  #+0.3
plt.plot(t_drive+time_delay[1], f_t1, 'o',  label='DRW2')  #+0.6
plt.plot(t_drive+time_delay[2] ,f_t2, 'o', label='DRW3')  #+0.9
plt.plot(t_drive+time_delay[3], f_t3, 'o', label='DRW4')  #+0.12
#plt.plot(t_s,f)

plt.xlabel('t (days)')
plt.ylabel('Fraction Variation')
plt.legend()
plt.show()

In [114]:
min_ = np.ceil(np.min(t_drive+time_delay[0]))
max_ = np.floor(np.max(t_drive+time_delay[2]))
# current sampling
t_s = np.sort(sample[0:5476])
np.savetxt('t_s.txt',t_s)
np.savetxt('f.txt',f)

In [124]:
# creating the desired sampling t_k ,
t_k_step = 8  # step for desired sampling
t_k_ = np.linspace(min_,max_,np.int(np.abs((min_-max_)/t_k_step)),dtype=int,retstep=True)
t_k = t_k_[0]
h = t_k_[1]
print('checking the step size', h)

checking the step size 8.022491349480969


In [125]:
with open('t_k_.pickle', 'wb') as g:
    pickle.dump(t_k_, g)

In [136]:
%run sinc.py

f_k shape (5476,)
f_k shape (579,)


In [184]:
%run gradient_descent.py

/u/zaidouni/Desktop/quasar_project/gradient_descent.py:41: RuntimeWarning: invalid value encountered in matmul
  X_new = X + mu*((A.T)@(Y-A@X))
